In [1]:
# import libraries we'll use
import spacy # fast NLP
import pandas as pd # dataframes
import langid # language identification (i.e. what language is this?)
from nltk.classify.textcat import TextCat # language identification from NLTK
from matplotlib.pyplot import plot # not as good as ggplot in R :p

In [2]:
AirData=pd.read_csv("reviews.csv")

In [3]:
AirData["comments"]=AirData["comments"].astype("str")

In [8]:
f=open("The_doc.txt",'a')
f.writelines(['\n', str(AirData["comments"])])
f.close()

In [9]:
ids_langid = AirData["comments"].apply(langid.classify)

# get just the language label
langs = ids_langid.apply(lambda tuple: tuple[0])

# how many unique language labels were applied?
print("Number of tagged languages (estimated):")
print(len(langs.unique()))

# percent of the total dataset in English
print("Percent of data in English (estimated):")
print((sum(langs=="en")/len(langs))*100)

Number of tagged languages (estimated):
49
Percent of data in English (estimated):
95.1476374041251


In [10]:
English_comment=AirData["comments"][langs=='en']

In [11]:
len(English_comment)

95768

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(English_comment.values.astype('U'))

In [13]:
doc_term_matrix

<95768x12472 sparse matrix of type '<class 'numpy.int64'>'
	with 1618138 stored elements in Compressed Sparse Row format>

In [14]:
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [15]:
first_topic = LDA.components_[0]

In [16]:
top_topic_words = first_topic.argsort()[-10:]

In [17]:
for i in top_topic_words:#words with highest probabilities
    print(count_vect.get_feature_names()[i])

good
host
house
clean
lovely
place
nice
great
stay
room


In [20]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
import pandas as pd
import gensim
import pyLDAvis.gensim

In [21]:
pattern = r'\b[^\d\W]+\b'
tokenizer = RegexpTokenizer(pattern)
en_stop = get_stop_words('en')
lemmatizer = WordNetLemmatizer()

In [25]:
# list for tokenized documents in loop
texts = []
for i in English_comment.iteritems():
    # clean and tokenize document string
    raw = str(i[1]).lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [raw for raw in tokens if not raw in en_stop]
    
    # remove stop words from tokens
    #stopped_tokens_new = [raw for raw in stopped_tokens if not raw in remove_words]
    
    # lemmatize tokens
    lemma_tokens = [lemmatizer.lemmatize(tokens) for tokens in stopped_tokens]
    
    # remove word containing only single char
    new_lemma_tokens = [raw for raw in lemma_tokens if not len(raw) == 1]
    
    # add tokens to list
    texts.append(new_lemma_tokens)

# sample data
print(texts[0])

['stay', 'marcus', 'bristol', 'fantastic', 'every', 'way', 'great', 'host', 'picking', 'bus', 'stop', 'recommending', 'place', 'try', 'leaving', 'plenty', 'pastry', 'breakfast', 'item', 'enjoy', 'morning', 'flat', 'modern', 'bright', 'clean', 'spacious', 'best', 'right', 'bristol', 'lovely', 'harbourside', 'will', 'definitely', 'stay', 'next', 'time', 're', 'bristol', 'thanks', 'marcus']


In [26]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [27]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=15, id2word = dictionary, passes=20)
import pprint
pprint.pprint(ldamodel.top_topics(corpus,topn=5))

[([(0.13236739, 'great'),
   (0.09151317, 'stay'),
   (0.07870538, 'location'),
   (0.06933419, 'place'),
   (0.035454422, 'lovely')],
  -1.117900633597799),
 ([(0.16229437, 'home'),
   (0.09399064, 'feel'),
   (0.078136645, 'made'),
   (0.068232656, 'welcome'),
   (0.05063699, 'host')],
  -1.167171974436044),
 ([(0.0637953, 'host'),
   (0.057270795, 'great'),
   (0.052149408, 'clean'),
   (0.046336748, 'recommend'),
   (0.04298988, 'room')],
  -1.3575514382791576),
 ([(0.07878998, 'touch'),
   (0.06279669, 'breakfast'),
   (0.041320503, 'coffee'),
   (0.033362906, 'tea'),
   (0.028643062, 'lovely')],
  -1.3729581681238427),
 ([(0.05552737, 'walk'),
   (0.04564894, 'centre'),
   (0.035886075, 'city'),
   (0.035134688, 'close'),
   (0.03029118, 'minute')],
  -1.5407864228030164),
 ([(0.045859598, 'stay'),
   (0.03133759, 'place'),
   (0.028275153, 'will'),
   (0.024770802, 'back'),
   (0.024391266, 'bristol')],
  -1.6266705230696548),
 ([(0.07814484, 'bristol'),
   (0.051285274, 'great'

In [28]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11    -0.075688 -0.110978       1        1  13.771910
12    -0.269440 -0.090382       2        1  10.895884
14    -0.262510 -0.077125       3        1  10.172887
7     -0.061568  0.196827       4        1   8.432131
6     -0.188196  0.089679       5        1   8.101104
5     -0.001512  0.171923       6        1   7.876177
0     -0.203640 -0.044949       7        1   7.559328
13     0.187200  0.181840       8        1   6.220492
9      0.198980  0.074554       9        1   5.174975
2      0.071148 -0.009717      10        1   4.643223
1     -0.070284  0.049855      11        1   4.481072
4      0.112256  0.138928      12        1   4.041103
3      0.042932 -0.254643      13        1   3.704656
10     0.276694 -0.160234      14        1   2.561509
8      0.243629 -0.155579      15        1   2.363548, topic_info=         Term          Freq         Total Category  logprob  loglift
11      great  50463.000000  50463.000000  Default  30.0000  30.0000
319      home  14163.000000  14163.000000  Default  29.0000  29.0000
100  location  32226.000000  32226.000000  Default  28.0000  28.0000
23      place  40328.000000  40328.000000  Default  27.0000  27.0000
102     night   8379.000000   8379.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
880    couple    394.699890   2404.241211  Topic15  -4.7262   1.9381
79       much    435.045410   5278.591797  Topic15  -4.6288   1.2490
213   perfect    455.242462  12297.257812  Topic15  -4.5835   0.4487
216     quiet    291.688965   8186.842285  Topic15  -5.0286   0.4104
141      work    224.725937   2210.177246  Topic15  -5.2894   1.4591

[896 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
472       1  0.598125    absolutely
472       2  0.136323    absolutely
472       3  0.060334    absolutely
472       7  0.204811    absolutely
1653     13  0.990929  accommodated
...     ...       ...           ...
168       1  0.995270           wow
1138      1  0.993730            xx
1852      8  0.996012         young
2020     13  0.994115         yummy
3258      7  0.992362           zoo

[1412 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 13, 15, 8, 7, 6, 1, 14, 10, 3, 2, 5, 4, 11, 9])